In [1]:
from accelerate import init_empty_weights
from transformers import AutoModelForCausalLM, AutoConfig
import os
from rich import print
os.environ['CUDA_VISIBLE_DEVICES']='1,2,3' # better be set at the start of the program

model_path = "/models/glm-4v-9b"
hf_config = AutoConfig.from_pretrained(model_path,
                                        trust_remote_code=True)

with init_empty_weights(): # This will accelerate 10x faster than normally from_config
    model = AutoModelForCausalLM.from_config(hf_config,
                                         trust_remote_code=True)
print(model) # model architecture with empty weights

## Remove llm modules
del model.transformer.embedding
del model.transformer.rotary_pos_emb
del model.transformer.encoder
del model.transformer.output_layer
## Remove llm modules


/home/gmcc/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(151552, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-39): 40 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (attn): SdpaAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_features=4096, out_features=151552, bias=False)
    (vision): EVA2CLIPModel(
      (patch_embedding): PatchEmbedding(
        (proj): Conv2d(3, 1792, kernel_size=(14, 14), stride=(14, 14))
        (position_embedding): Embedding(6401, 1792)
      )
      (transformer): Transformer(
        (layers): ModuleList(
          (0-62): 63 x TransformerLayer(
            (input_layernorm): LayerNorm((1792,), eps=1e-06, elementwise_affine=True)
            (attention): Attention(
              (query_key_value): Linear(in_features=1792, out_features=5376, bias=True)
              (dense): Linear(in_features=1792, out_features=1792, bias=True)
              (output_dropout): Dropout(p=0.0, inplace=False)
            )
            (mlp): MLP(
              (activation_fn): GELUActivation()
              (fc1): Linear(in_features=1792, out_features=15360, bias=True)
              (fc2): Linear(in_features=15360, out_features=1792, bias=True)
            )
            (post_attention_layernorm): LayerNorm((1792,), eps=1e-06, elementwise_affine=True)
          )
        )
      )
      (linear_proj): GLU(
        (linear_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (norm1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (act1): GELU(approximate='none')
        (dense_h_to_4h): Linear(in_features=4096, out_features=13696, bias=False)
        (gate_proj): Linear(in_features=4096, out_features=13696, bias=False)
        (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
      )
      (conv): Conv2d(1792, 4096, kernel_size=(2, 2), stride=(2, 2))
    )
  )
)

In [5]:
from accelerate import  infer_auto_device_map
import torch

# affected by CUDA_VISIBLE_DEVICES and only affected from first initialized
print("CUDA device_count:", torch.cuda.device_count())
max_memory = { i: torch.cuda.mem_get_info(i)[0] for i in range(torch.cuda.device_count()) }
max_memory[0] = max_memory[0]//16 # decrease the first card usage
print(max_memory) # {0: 3354591232//16, 1: 22564503552, 2: 22564503552}

# vision transformer layers.
# If not specified, some modules in a layer may be splitted to different cards.
no_split_module_classes = ['TransformerLayer']
device_map = infer_auto_device_map(
    model,
    no_split_module_classes=no_split_module_classes,
    dtype=torch.float16,
    max_memory=max_memory
)
print(device_map) # OrderedDict([('transformer', 0)])

torch.cuda.empty_cache()
print("real CUDA memory_allocated: ",torch.cuda.memory_allocated()) # nvidia-smi lies

same_device_keys = [('transformer.vision.linear_proj',
                     'transformer.vision.boi',  
                     'transformer.vision.eoi')]
print(type(model.transformer.vision.boi)) # torch.nn.Parameter

for keys in same_device_keys:
    pass


CUDA device_count: 3

{0: 1410281472, 1: 22564503552, 2: 22564503552}

OrderedDict([('transformer.vision.boi', 0), ('transformer.vision.eoi', 0), ('transformer.vision.patch_embedding', 
0), ('transformer.vision.transformer.layers.0', 0), ('transformer.vision.transformer.layers.1', 0), 
('transformer.vision.transformer.layers.2', 0), ('transformer.vision.transformer.layers.3', 0), 
('transformer.vision.transformer.layers.4', 0), ('transformer.vision.transformer.layers.5', 0), 
('transformer.vision.transformer.layers.6', 0), ('transformer.vision.transformer.layers.7', 0), 
('transformer.vision.transformer.layers.8', 0), ('transformer.vision.transformer.layers.9', 1), 
('transformer.vision.transformer.layers.10', 1), ('transformer.vision.transformer.layers.11', 1), 
('transformer.vision.transformer.layers.12', 1), ('transformer.vision.transformer.layers.13', 1), 
('transformer.vision.transformer.layers.14', 1), ('transformer.vision.transformer.layers.15', 1), 
('transformer.vision.transformer.layers.16', 1), ('transformer.vision.transformer.layers.17', 1), 
('transformer.vision.transformer.layers.18', 1), ('transformer.vision.transformer.layers.19', 1), 
('transformer.vision.transformer.layers.20', 1), ('transformer.vision.transformer.layers.21', 1), 
('transformer.vision.transformer.layers.22', 1), ('transformer.vision.transformer.layers.23', 1), 
('transformer.vision.transformer.layers.24', 1), ('transformer.vision.transformer.layers.25', 1), 
('transformer.vision.transformer.layers.26', 1), ('transformer.vision.transformer.layers.27', 1), 
('transformer.vision.transformer.layers.28', 1), ('transformer.vision.transformer.layers.29', 1), 
('transformer.vision.transformer.layers.30', 1), ('transformer.vision.transformer.layers.31', 1), 
('transformer.vision.transformer.layers.32', 1), ('transformer.vision.transformer.layers.33', 1), 
('transformer.vision.transformer.layers.34', 1), ('transformer.vision.transformer.layers.35', 1), 
('transformer.vision.transformer.layers.36', 1), ('transformer.vision.transformer.layers.37', 1), 
('transformer.vision.transformer.layers.38', 1), ('transformer.vision.transformer.layers.39', 1), 
('transformer.vision.transformer.layers.40', 1), ('transformer.vision.transformer.layers.41', 1), 
('transformer.vision.transformer.layers.42', 1), ('transformer.vision.transformer.layers.43', 1), 
('transformer.vision.transformer.layers.44', 1), ('transformer.vision.transformer.layers.45', 1), 
('transformer.vision.transformer.layers.46', 1), ('transformer.vision.transformer.layers.47', 1), 
('transformer.vision.transformer.layers.48', 1), ('transformer.vision.transformer.layers.49', 1), 
('transformer.vision.transformer.layers.50', 1), ('transformer.vision.transformer.layers.51', 1), 
('transformer.vision.transformer.layers.52', 1), ('transformer.vision.transformer.layers.53', 1), 
('transformer.vision.transformer.layers.54', 1), ('transformer.vision.transformer.layers.55', 1), 
('transformer.vision.transformer.layers.56', 1), ('transformer.vision.transformer.layers.57', 1), 
('transformer.vision.transformer.layers.58', 1), ('transformer.vision.transformer.layers.59', 1), 
('transformer.vision.transformer.layers.60', 1), ('transformer.vision.transformer.layers.61', 1), 
('transformer.vision.transformer.layers.62', 1), ('transformer.vision.linear_proj', 1), ('transformer.vision.conv',
1)])

real CUDA memory_allocated:  0

<class 'torch.nn.parameter.Parameter'>